In [1]:
import requests

# Get API auth token

In [10]:
from getpass import getpass

username = "jochen"
password = getpass()

········


In [12]:
token_path = reverse("api-token-auth")
token_url = f"http://localhost:8000{token_path}"
token_url = f"https://python-podcast.staging.django-cast.com{token_path}"
print(token_url)

r = requests.post(token_url, data={"username": username, "password": password})
token = r.json()["token"]
print(token)

https://python-podcast.staging.django-cast.com/api/api-token-auth/
4acf2cddb07f66ceaf8bec25022ce4b2258fe5eb


# Get Training Data

In [17]:
path = reverse("cast:api:comment_training_data")
url = f"http://localhost:8000{path}"
url = f"https://python-podcast.staging.django-cast.com/api/comment_training_data/"
print(url)

https://python-podcast.staging.django-cast.com/api/comment_training_data/


In [31]:
%%time
headers = {"Authorization": f"Token {token}"}
r = requests.get(url, headers=headers)
messages = r.json()

CPU times: user 63.4 ms, sys: 10.2 ms, total: 73.6 ms
Wall time: 344 ms


In [32]:
len(messages)

1711

# Train Spamfilter

In [33]:
from cast.models.moderation import NaiveBayes

In [24]:
%%time
nb = NaiveBayes().fit(train)

CPU times: user 107 ms, sys: 5.03 ms, total: 112 ms
Wall time: 109 ms


In [27]:
label, message = train[23]
print(label, nb.predict_label(message))

ham ham


# Split Train/Test

In [71]:
def split_into_folds(items, folds):
    k, m = divmod(len(items), folds)
    return [items[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(folds)]


def flatten(items):
    return [item for sublist in items for item in sublist]



def generate_train_test(folds):
    for i, n in enumerate(folds):
        all_but_n = flatten(folds[:i]) + flatten(folds[i + 1:])
        yield n, all_but_n

## Split Into Spam and Ham

To be able to sample stratified.

In [106]:
spam, ham = [], []
for label, msg in messages:
    if label == "ham":
        ham.append((label, msg))
    else:
        spam.append((label, msg))
random.shuffle(spam)
random.shuffle(ham)
print(len(spam), len(ham))

1589 122


In [108]:
num_folds = 3
ham_folds = split_into_folds(ham, num_folds)
spam_folds = split_into_folds(spam, num_folds)
folds = []
for ham_fold, spam_fold in zip(ham_folds, spam_folds):
    folds.append(ham_fold + spam_fold)

In [84]:
random.shuffle(messages)
folds = split_into_folds(messages, 3)

In [109]:
for test, train in generate_train_test(folds):
    print(len(test), len(train))

571 1140
571 1140
569 1142


# Evaluate

In [110]:
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)

    true_positives = 0
    all_observations = len(test)

    for label, message in test:
        if nb.predict_label(message) == label:
            true_positives += 1

    accuracy = true_positives / all_observations
    print(f"Accuracy: {accuracy:.3f}", true_positives, all_observations)

571 1140
Accuracy: 0.802 458 571
571 1140
Accuracy: 0.809 462 571
569 1142
Accuracy: 0.772 439 569


In [111]:
%%time

train_messages, test_messages = split_train_test(messages)
nb = NaiveBayes().fit(train_messages)
true_positives = 0
all_observations = len(test_messages)

for label, message in test_messages:
    if nb.predict_label(message) == label:
        true_positives += 1

accuracy = true_positives / all_observations
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.761
CPU times: user 136 ms, sys: 3.1 ms, total: 139 ms
Wall time: 137 ms


In [112]:
def generate_outcomes(nb, test_messages):
    outcomes = (("true", "false"), ("positive", "negative"))
    possible_results = [f"{a}_{b}" for b in outcomes[1] for a in outcomes[0]]
    result_template = dict.fromkeys(possible_results, 0)

    labels = set(nb.prior_probabilities)
    label_results = {label: dict(result_template) for label in labels}
    all_observations = len(test_messages)

    for label, message in test_messages:
        predicted = nb.predict_label(message)
        if label == predicted:
            label_results[label]["true_positive"] += 1
        else:
            label_results[label]["false_negative"] += 1
            label_results[predicted]["false_positive"] += 1
    return label_results

In [113]:
def precision_recall_f1(result):
    all_observations = sum(result.values())
    tp = result["true_positive"]
    fp = result["false_positive"]
    fn = result["false_negative"]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1


def show_result(label_results):
    for label, result in label_results.items():
        precision, recall, f1 = precision_recall_f1(result)
        print(
            f"{label: >4} f1: {f1:.3f} precision: {precision:.3f} recall: {recall:.3f}"
        )

In [114]:
results = None
for test, train in generate_train_test(folds):
    print(len(test), len(train))
    nb = NaiveBayes().fit(train)
    label_result = generate_outcomes(nb, test)
    if results is None:
        results = label_result
    else:
        for label, counts in label_result.items():
            for name, count in counts.items():
                results[label][name] += count
show_result(results)

571 1140
571 1140
569 1142
spam f1: 0.876 precision: 0.999 recall: 0.779
 ham f1: 0.407 precision: 0.256 recall: 0.992


In [115]:
nb.prior_probabilities

{'ham': 0.07180385288966724, 'spam': 0.9281961471103327}

In [98]:
count = 0
for label, message in messages:
    if label == "ham":
        print(message)
        count += 1
print(count)

Jürgen  Nach drei Wochen Pendeln hatte ich die Folge durch 😅

Nachdem ich in den letzten Jahren fast nur mit ZODB gearbeitet habe, habe ich durch die Folge viel Neues gelernt! Vor allem die Geschichte mit Postgres und JSON hört sich ja super an.

Vielen Dank und immer weiter so!
Norbert  Danke für diese sehr interessante Folge!

Für mich waren viele nutzbare Infos dabei, die ich entweder noch gar nicht wusste oder jedenfalls nicht derart detailliert.

Gruss
Norbert  War interessant für mich, danke!
Jürgen  DNS over https im Firefox kommt afaik nur in den USA.
Johannes  Voll die gute Episode!
Jochen  Danke :)
Jochen  Nee, ist gut. Hilft dabei uns dazu zu bringen, dass das hier mal weitergeht :).
Jochen Wersdörfer  Also ich mache das vor allem auch, um selbst etwas dabei über das Thema zu lernen. Und öffentlich failen hat halt einen ganz guten Lerneffekt :). Aber ja, vielleicht sollten wir uns zumindest bei Themen, mit denen wir uns nicht eh schon jeden Tag beschäftigen etwas besser vorb

Ajo, wieder eine klasse Episode! Hoffe bald auf Nachschub!
Alpengreis  Wieder einiges gelernt, danke!
Jochen  Oh, wenn du eine gute Hostinglösung gefunden hast, sag bescheid :).

Hmm, jo - also zum Verschieben von Dateien nehme ich meistens scp/rsync. Wofür möchtest du den Dateien zwischen Rechnern verteilen? Vielleicht gibt es für deinen Anwendungsfall ja auch noch andere Möglichkeiten...
Dominik  oh, jetzt habe ich aus Versehen einen Kommentar gelöscht der sich auf die letzte Folge bezog.. in dem stand: "*piep, *piep, *piep"
da ging sofort mein Spam-Filter an :-----P
Jochen Wersdörfer  Hallo Philip,

vielen Dank für deinen Hinweis. Ja, Adressen eine normalisierte Form zu bringen, ist irgendwie immer schwieriger als man denkt. Je nachdem wofür man den Ort verwenden möchte, kann man das vielleicht  schon so machen, denn vielleicht braucht man bei einer Marketingkampagne gar nicht den Ortsnamen, sondern halt nur die ungefähre Region - aber es hängt halt immer davon ab, was der Zweck der

pi = 3.142159265358979323846264338327950288419716939937510582097494459230781640628620899862803482534211706798214808651328230664709384460955058223172535940812848111745028410270193852110555964462294895493038196442881097566593344612847564823378678316527120190914564856692346034861045432664821339360726024914127372458700660631558817488152092096282925409171536436789259036001133053054882046652138414695194151160943305727036575959195309218611738193261179310511854807446237996274956735188575272489122793818301194912983367336244065664308602139494639522473719070217986094370277053921717629317675238467481846766940513200056812714526356082778577134275778960917363717872146844090122495343014654958537105079227968925892354201995611212902196086403441815981362977477130996051870721134999999837297804995105973173281609631859502445945534690830264252230825334468503526193118817101000313783875288658753320838142061717766914730359825349042875546873115956286388235378759375195778185778053217122680661300192787661119590921

Da braucht man die Klasse natürlich für. Danke auch :).
Jochen Wersdörfer  Vielen Dank - ja, waren einige ausführliche Abschweifungen dabei diesmal. Aber es gab halt auch viel zu erzählen 😇.

Dein Kommentar war übrigens auch der erste, der nach dem Umstieg auf wagtail nicht vom Spamfilter abgelehnt wurde. Gut, dass das mal jemand getestet hat 😅.
Johannes  Stimmt, da haben wir einen Fehler gemacht. Danke für das Ausprobieren und die Korrektur!
Frank Mankel  Hallo Podcast Team,

ich finde es absolut rekordverdächtig, Euer Intro hat diesmal fast 1 1/2 Stunden gedauert. Meinen Respekt dafür ;)

Spaß beiseite, ich mag Euren Podcast und freue mich auf jede Folge. Ok, ab und zu bin ich von den Themen überfordert (Python Einsteiger) aber ich habe schon einiges bei Euch gelernt.

Danke für den tollen Podcast!

Gruß
FrankM
Jochen Wersdörfer  Schön zu hören :).
Alpengreis  Ah ja, eine EDIT-Funktion für die Beiträge hier wäre auch noch schön :-)
Frank  ... bei dem Gestammel zu Vue habe ich dann ab

Norbert
Dominik  Hey Dirk, Danke für die Links und sorry, für die verspätete Antwort - hier geht es gerade drunter und drüber.
Besonders die letzte der Definitionen von DevOps ist sehr interessant - es entwickelt & verflechtet sich quasi gemeinsam ineinander.
Hatten die Links ein Ablaufdatum? - letzte Woche konnte ich sie noch sehen, heute sind sie nicht mehr verfügbar. Ich fände es als "anderer Leser" total spannend, wenn ich die Links hier ebenfalls angucken könnte.

Ich schaffe es leider nicht auf die FrOSCon, Jochen war afaik auf "vielleicht"

Gruß,
Dominik
Domi  *Hüsterchen https://www.youtube.com/watch?v=WVDQEoe6ZWY
Alpengreis  Ja, Du hast schon recht, dass mit dem gleichen Integer reproduzierbare Ergebnisse "forciert" werden, wie z.B. mit:

import random

for i in range(2):
    random.seed(2020)
    for i in range(20):
        print(random.random())
    print()


Im Gegensatz zu Folgendem:

import random

for i in range(2):
    random.seed()
    for i in range(20):
        print